In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import swing_technical_indicators
import numpy as np


# Replace with your Alpha Vantage API key
api_key = '89PMD611NPUTS89W'

# Function to fetch forex data
def fetch_forex_data(from_currency, to_currency):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'FX_DAILY',
        'from_symbol': from_currency,
        'to_symbol': to_currency,
        'apikey': api_key,
        'outputsize': 'full'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extracting the time series data
    time_series = data.get('Time Series FX (Daily)', {})
    
    # Converting to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Adding a column for the currency pair
    df['Currency Pair'] = f'{from_currency}/{to_currency}'
    
    # Filtering data for the last 10 years
    ten_years_ago = datetime.now() - timedelta(days=365*20)
    df = df[df.index >= ten_years_ago]
    
    return df

# List of common currency pairs to fetch which are allowed to trade in india 
currency_pairs = [
    ('USD', 'INR'), ('EUR', 'USD'), ('GBP', 'USD'), ('USD', 'JPY'), ('EUR', 'INR'),
    ('JPY', 'INR'), ('GBP', 'INR')
]

# Fetching data for all currency pairs and combining into a single DataFrame
all_data = pd.DataFrame()
for from_currency, to_currency in currency_pairs:
    forex_data = fetch_forex_data(from_currency, to_currency)
    all_data = pd.concat([all_data, forex_data])

# Resetting index for the combined DataFrame
all_data.reset_index(inplace=True)
all_data.rename(columns={'index': 'Date',"1. open":"Open_price","2. high":"Day_high","3. low":"Day_low","4. close":"Closing_price"}, inplace=True)

print(all_data)

            Date Open_price   Day_high    Day_low Closing_price Currency Pair
0     2014-11-07   61.39000   61.62000   61.34000      61.40000       USD/INR
1     2014-11-10   61.50000   61.63500   61.34000      61.49500       USD/INR
2     2014-11-11   61.53000   61.55500   61.50500      61.50800       USD/INR
3     2014-11-12   61.50800   61.56000   61.35000      61.39100       USD/INR
4     2014-11-13   61.36800   61.62300   61.35000      61.56500       USD/INR
...          ...        ...        ...        ...           ...           ...
18295 2024-11-21  106.69620  106.87660  106.21990     106.29930       GBP/INR
18296 2024-11-22  106.34520  106.38450  105.44460     105.74120       GBP/INR
18297 2024-11-25  106.27350  106.39990  105.62610     105.80810       GBP/INR
18298 2024-11-26  105.84890  106.23050  105.42730     105.87820       GBP/INR
18299 2024-11-27  105.96210  106.50110  105.92090     106.39840       GBP/INR

[18300 rows x 6 columns]


In [5]:
all_data.tail()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair
18295,2024-11-21,106.69620,106.87660,106.21990,106.29930,GBP/INR
18296,2024-11-22,106.34520,106.38450,105.44460,105.74120,GBP/INR
18297,2024-11-25,106.27350,106.39990,105.62610,105.80810,GBP/INR
18298,2024-11-26,105.84890,106.23050,105.42730,105.87820,GBP/INR
18299,2024-11-27,105.96210,106.50110,105.92090,106.39840,GBP/INR


In [3]:
usdinr = all_data.loc[all_data['Currency Pair']=='USD/INR']
usdinr.reset_index(drop=True,inplace=True)

In [4]:
usdinr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           2623 non-null   datetime64[ns]
 1   Open_price     2623 non-null   object        
 2   Day_high       2623 non-null   object        
 3   Day_low        2623 non-null   object        
 4   Closing_price  2623 non-null   object        
 5   Currency Pair  2623 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 123.1+ KB


In [6]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

usdinr["Open_price"] = usdinr["Open_price"].astype(float)
usdinr["Day_high"] = usdinr["Day_high"].astype(float)
usdinr["Day_low"] = usdinr["Day_low"].astype(float)
usdinr["Closing_price"] = usdinr["Closing_price"].astype(float)

Visualization of Trend

In [7]:
import plotly.express as px

# Subset the relevant columns
a = usdinr[["Date", "Open_price", "Currency Pair"]]

# Get the single currency pair value
currency_pair = usdinr['Currency Pair'].iloc[0]  # Assumes all rows have the same value

# Plot using plotly.express.line
fig = px.line(a, x='Date', y='Open_price', 
              title=f'Open Price for last 10 years for the currency pair: {currency_pair}')

# Display the plot
fig.show()

In [22]:
import plotly.express as px

# Subset the relevant columns
a = usdinr[["Date", "Closing_price", "Currency Pair"]]

# Get the single currency pair value
currency_pair = usdinr['Currency Pair'].iloc[0]  # Assumes all rows have the same value

# Plot using plotly.express.line
fig = px.line(a, x='Date', y='Closing_price', 
              title=f'Closing Price for last 10 years for the currency pair: {currency_pair}')

# Display the plot
fig.show()

AdFuller's stationarity test

In [8]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):
    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    print("Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary")

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

for i in usdinr.columns : 
  if i!='Currency Pair' :
    print("ADFuller test for column : ",i)
    check_stationarity(usdinr[i])

ADFuller test for column :  Date
ADF Statistic: 0.607639
p-value: 0.987802
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary
Non-stationary
ADFuller test for column :  Open_price
ADF Statistic: -0.718840
p-value: 0.841809
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary
Non-stationary
ADFuller test for column :  Day_high
ADF Statistic: -0.722567
p-value: 0.840817
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary
Non-stationary
ADFuller test for column :  Day_low
ADF Statistic: -0.601546
p-value: 0.870632
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary
Non-stationary
ADFuller test fo

Seasonality Decomposition

In [9]:
from statsmodels.tsa.seasonal import seasonal_decompose

def deseasonalize_column(data, column, period=7):
    # Decompose the column
    decomposition = seasonal_decompose(data[column], model='additive', period=period, extrapolate_trend='freq')
    
    # Extract the residual component as deseasonalized data
    deseasonalized = data[column] - decomposition.seasonal
    
    return deseasonalized

# Identify non-stationary columns
non_stationary_columns = ['Day_high', 'Day_low', 'Open_price']

# Create deseasonalized columns
for col in non_stationary_columns:
    deseasonalized_col = deseasonalize_column(usdinr, col, period=7)  # Adjust the period as needed
    usdinr[f'Deseasonalized_{col}'] = deseasonalized_col

In [10]:
usdinr.head()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Deseasonalized_Day_high,Deseasonalized_Day_low,Deseasonalized_Open_price
0,2014-11-07,61.390,61.620,61.340,61.400,USD/INR,61.610101,61.341917,61.393506
1,2014-11-10,61.500,61.635,61.340,61.495,USD/INR,61.627339,61.332157,61.493596
2,2014-11-11,61.530,61.555,61.505,61.508,USD/INR,61.547724,61.499491,61.516284
3,2014-11-12,61.508,61.560,61.350,61.391,USD/INR,61.567341,61.356586,61.504031
4,2014-11-13,61.368,61.623,61.350,61.565,USD/INR,61.633775,61.348468,61.385332


Add EMA Features

In [11]:
usdinr['EMA_50'] = swing_technical_indicators.calculate_100ema(usdinr, 'Closing_price', 50)
usdinr['EMA_100'] = swing_technical_indicators.calculate_100ema(usdinr, 'Closing_price', 100)
usdinr['EMA_200'] = swing_technical_indicators.calculate_200ema(usdinr, 'Closing_price', 200)

Add RSI

In [12]:
usdinr['RSI'] = swing_technical_indicators.calculate_rsi(usdinr,'Closing_price',21)

Add MACD Features

In [13]:
usdinr_macd = swing_technical_indicators.calculate_macd(usdinr, column_name='Closing_price', short_period=12, long_period=26, signal_period=9)
for i in usdinr_macd.columns:
    usdinr[f"{i}"] = usdinr_macd[f'{i}']

Add Bollinger Bands

In [14]:
bollinger_data = swing_technical_indicators.calculate_bollinger_bands(usdinr['Closing_price'])
usdinr = usdinr.join(bollinger_data)
usdinr.drop('Price',axis=1, inplace=True)

In [15]:
usdinr.head()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Deseasonalized_Day_high,Deseasonalized_Day_low,Deseasonalized_Open_price,EMA_50,EMA_100,EMA_200,RSI,MACD,Signal_Line,MACD_Histogram,SMA,Upper Band,Lower Band
0,2014-11-07,61.390,61.620,61.340,61.400,USD/INR,61.610101,61.341917,61.393506,61.400000,61.400000,61.400000,0.0,0.000000,0.000000,0.000000,61.7496,62.164424,61.334776
1,2014-11-10,61.500,61.635,61.340,61.495,USD/INR,61.627339,61.332157,61.493596,61.403725,61.401881,61.400945,0.0,0.007578,0.001516,0.006063,61.7496,62.164424,61.334776
2,2014-11-11,61.530,61.555,61.505,61.508,USD/INR,61.547724,61.499491,61.516284,61.407815,61.403983,61.402010,0.0,0.014466,0.004106,0.010361,61.7496,62.164424,61.334776
3,2014-11-12,61.508,61.560,61.350,61.391,USD/INR,61.567341,61.356586,61.504031,61.407155,61.403725,61.401901,0.0,0.010365,0.005358,0.005007,61.7496,62.164424,61.334776
4,2014-11-13,61.368,61.623,61.350,61.565,USD/INR,61.633775,61.348468,61.385332,61.413345,61.406919,61.403524,0.0,0.020914,0.008469,0.012445,61.7496,62.164424,61.334776


In [16]:
X = usdinr.drop(['Date','Open_price','Day_high','Day_low','Closing_price','Currency Pair'],axis=1)
columns = X.columns
y =usdinr['Closing_price']

In [18]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X,columns = columns)

In [19]:
usdinr.to_csv('USDINR.csv')
X.to_csv('X_USDINR.csv')
y.to_csv('Y_USDINR.csv')